# 1- Initialization

## 1.1- Setup environment

In [49]:
# upgrade pip
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [50]:
# install tfx
!pip install -q -U --use-feature=2020-resolver tfx

# If you are using Google Colab, the first time that you run the cell above,  
# you must restart the runtime (Runtime > Restart runtime ...). This is because of the way that Colab loads packages.

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.

Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --use-feature


## 1.2- Import packages

In [51]:
import os
import tempfile
import absl
import pprint

import tensorflow as tf
import tfx
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

pp = pprint.PrettyPrinter()

TensorFlow version: 2.3.1
TFX version: 0.21.2


## 1.3- Setup pipeline paths

In [52]:
#setup pipeline paths

_data_root = "fraud_data"

# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Chicago Taxi Pipeline example.
_taxi_root = os.path.join(_tfx_root, 'examples/creditcard')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/creditcard')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

## 1.4- Download credit card fraud dataset

In [53]:
# load credit card fraud dataset (source: https://www.kaggle.com/mlg-ulb/creditcardfraud) if not exists in data folder

import urllib.request
if not os.path.isfile(_data_root + "/creditcard.csv"):
    if not os.path.exists(_data_root): os.mkdir(_data_root)
    urllib.request.urlretrieve("https://uplanet-archives.s3.amazonaws.com/creditcard.csv", _data_root + "/creditcard.csv")
!head {_data_root + "/creditcard.csv"}

Time,V01,V02,V03,V04,V05,V06,V07,V08,V09,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62,"0"
0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783

## 1.5- setup context

In [54]:
# Here, we create an InteractiveContext using default parameters. This will
# use a temporary directory with an ephemeral ML Metadata database instance.
# To use your own pipeline root or database, the optional properties
# `pipeline_root` and `metadata_connection_config` may be passed to
# InteractiveContext. Calls to InteractiveContext are no-ops outside of the
# notebook.
context = InteractiveContext()

# 2- Processing Pipeline

## 2.1- ExampleGen
Load data into the pipeline and split dataset into train and evaluation data
* Input: consumes daya from external data sources
* Output: tf.Exemple records

In [55]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Processing input csv data fraud_data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


["train", "eval"] /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/CsvExampleGen/examples/1


## 2.2- StatisticsGen
Generate dataset statistics
* Input: tf.Example records from the ExampleGen component
* Output: dataset statistics

In [56]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Using 1 process(es) for Beam pipeline execution.
INFO:absl:Generating statistics for split train
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/StatisticsGen/statistics/2/eval.
/root/miniconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow_data_validation/arrow/arrow_util.py:239: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
  types.FeaturePath([column_name]), column.data.chunk(0), weights):
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


## 2.3- SchemaGen
Generates a schema by inferring types, categories, and ranges from the training data
* Input: dataset statistics from the StatisticsGen component
* Output: dataset schema

In [57]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Infering schema from statistics.
INFO:absl:Schema written to /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


,Type,Presence,Valency,Domain
Feature name,,,,
'Amount',FLOAT,required,single,-
'Class',INT,required,single,-
'Time',FLOAT,required,single,-
'V01',FLOAT,required,single,-
'V02',FLOAT,required,single,-
'V03',FLOAT,required,single,-
'V04',FLOAT,required,single,-
'V05',FLOAT,required,single,-
'V06',FLOAT,required,single,-


## 2.4- ExampleValidator
Looks for anomalies and missing values in the dataset
* Input: dataset schema from the SchemaGen component and data statistics from the StatisticsGen component
* Output: validation results

In [58]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics.
INFO:absl:Validation complete. Anomalies written to /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/ExampleValidator/anomalies/4.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


## 2.5- Transform
Performs feature engineering on the dataset.
* Input: 
    * tf.Examples from an ExampleGen component
    * data schema from a SchemaGen component.
* Output: 
    * transformed examples
    * transformed graph

In [59]:
_transform_module_file = 'creditcard_fraud_transform.py'

In [60]:
%%writefile {_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

_DENSE_FLOAT_FEATURE_KEYS = ['Amount', 'Time',
                             'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09',
                             'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
                             'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']
_LABEL_KEY = 'Class'

def preprocessing_fn(inputs):
    outputs = {}

    for key in _DENSE_FLOAT_FEATURE_KEYS:
      outputs[_transformed_name(key)] = tft.scale_to_z_score(
          _fill_in_missing(inputs[key]))
    
    outputs[_transformed_name(_LABEL_KEY)] = _fill_in_missing(inputs[_LABEL_KEY])

    return outputs

def _transformed_name(key):
  return key + '_xf'

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
  Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
  """
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Overwriting creditcard_fraud_transform.py


In [61]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_transform_module_file))
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Transform
INFO:absl:Using 1 process(es) for Beam pipeline execution.


INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/Transform/transform_graph/5/.temp_path/tftransform_tmp/b0987c06ae6846169e6f7695054db632/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
Type is unsupported, or the types of the items don't match field type in CollectionDef. Note this is a warning and probably safe to ignore.
'Counter' object has no attribute 'name'
INFO:tensorflow:SavedModel written to: /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/Transform/transform_graph/5/.temp_path/tftransform_tmp/50b62d066c2641428ea45f2759e6bcd7/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
I

INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(type_name: TransformGraph, uri: /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/Transform/transform_graph/5, id: 6)]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(type_name: Examples, uri: /tmp/tfx-interactive-2020-10-24T16_27_49.918036-9ptqf4bb/Transform/transformed_examples/5, id: 7)]
        ))

### 2.5.1 show transforme example record

In [62]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
for tfrecord in dataset.take(1):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "Amount_xf"
    value {
      float_list {
        value: 0.23524299263954163
      }
    }
  }
  feature {
    key: "Class_xf"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Time_xf"
    value {
      float_list {
        value: -1.9971795082092285
      }
    }
  }
  feature {
    key: "V01_xf"
    value {
      float_list {
        value: -0.6925919055938721
      }
    }
  }
  feature {
    key: "V02_xf"
    value {
      float_list {
        value: -0.04350024461746216
      }
    }
  }
  feature {
    key: "V03_xf"
    value {
      float_list {
        value: 1.666440725326538
      }
    }
  }
  feature {
    key: "V04_xf"
    value {
      float_list {
        value: 0.9690198302268982
      }
    }
  }
  feature {
    key: "V05_xf"
    value {
      float_list {
        value: -0.24057945609092712
      }
    }
  }
  feature {
    key: "V06_xf"
    value {
      float_list {
        value: 0.343514621257

## 2.6 Trainer
Trains the model
* Input: 
    * tf.Examples from ExempleGen component for training and eval
    * transform graph from the Transform component
    * data schema created by the SchemaGen component
* Output: model

### 2.6.1 Create Trainer class

In [63]:
_trainer_module_file = 'credit_card_fraud_trainer.py'

In [64]:
%%writefile {_trainer_module_file}
from typing import List, Text

import tensorflow as tf
import tensorflow_transform as tft
from tfx.components.trainer.executor import TrainerFnArgs
from tensorflow.keras import layers, models, optimizers
import os
import glob
import codecs
import numpy as np

_DENSE_FLOAT_FEATURE_KEYS = ['Amount', 'Time',
                             'V01', 'V02', 'V03', 'V04', 'V05', 'V06', 'V07', 'V08', 'V09',
                             'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19',
                             'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28']
_LABEL_KEY = 'Class'


# TFX Trainer will call this function.
def run_fn(fn_args: TrainerFnArgs):
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
  train_dataset, train_labels = _input_fn(fn_args.train_files, tf_transform_output, 40)
  eval_dataset, eval_labels = _input_fn(fn_args.eval_files, tf_transform_output, 40)
    
  #create model
  model = models.Sequential()
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(10, activation='relu'))
  model.add(layers.Dense(1, activation='relu'))
    
  model.compile(optimizer=optimizers.Adam(0.01), loss='MSE', metrics=['accuracy'])
    
  # train model
  log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq='batch')
  #model.fit(
  #  train_dataset,
  #  train_labels, 
  #   steps_per_epoch=fn_args.train_steps,
  #  validation_data=(eval_dataset, eval_labels), 
  #  validation_steps=fn_args.eval_steps,
  #  callbacks=[tensorboard_callback])
  model.fit(train_dataset, train_labels, validation_data=(eval_dataset, eval_labels), epochs=10)
    
  #save model
  #model.save("model.h5")
  model.save(fn_args.serving_model_dir, save_format='tf')

def _gzip_reader_fn(filenames):
  """Small utility returning a record reader that can read gzip'ed files."""
  return tf.data.TFRecordDataset(
      filenames,
      compression_type='GZIP')
    
def _input_fn(file_pattern: List[Text],
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    Features and labels Numpy arrays.
  """
  transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

  files = glob.glob(file_pattern[0])
  dataset = _gzip_reader_fn(files)

  dataset = dataset.map(lambda x: tf.io.parse_example(x, transformed_feature_spec))

  features = []
  labels = []
  for record in dataset:
    feature_row = []
    for feat in transformed_feature_spec:
        if feat in _transformed_names(_DENSE_FLOAT_FEATURE_KEYS):
            feature_row.append(record[feat].numpy())
        if feat == _transformed_name(_LABEL_KEY):
            labels.append(record[feat].numpy())
    features.append(feature_row)

  return np.array(features), np.array(labels)

def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]

def _transformed_name(key):
  return key + '_xf'

Overwriting credit_card_fraud_trainer.py


### 2.6.2 Train the model

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=5),
    eval_args=trainer_pb2.EvalArgs(num_steps=5))
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.


### 2.6.3 Analyze the model with Tensorboard

In [45]:
model_run_artifact_dir = trainer.outputs['model'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
